<a href="https://colab.research.google.com/github/Peheppy/YOLO-Coin-Detector/blob/main/model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

## Install Dependencies

In [ ]:
!pip install ultralytics
!pip install scikit-learn
!pip install matplotlib
!pip install pandas
!pip install pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.0 MB/s eta 0:00:00


## Import Libraries

In [ ]:
import os
import glob
import shutil
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from google.colab import files
from ultralytics import YOLO
import yaml
from google.colab import drive
drive.mount('/content/drive')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive


# Prepare Dataset

## Upload and Extract Dataset

In [ ]:
dataset_zip = "/content/drive/MyDrive/Projetos/YOLO-Coin/dataset.zip"

os.makedirs("datasets", exist_ok=True)

!unzip -q "{dataset_zip}" -d datasets/

!ls datasets/

classes.txt  images  labels  notes.json


## Split Data Using K-Fold

In [ ]:
K = 5

In [ ]:
images_path = "datasets/images"
labels_path = "datasets/labels"

exts = ["*.jpg", "*.jpeg", "*.png"]
all_images = []

for ext in exts:
    all_images.extend(glob.glob(os.path.join(images_path, ext)))

all_images = sorted(all_images)

# Read class names
with open("datasets/classes.txt", "r") as f:
    classes = [c.strip() for c in f.readlines()]

# Create base folder for folds
os.makedirs("datasets/folds", exist_ok=True)

fold_num = 1
for train_idx, val_idx in KFold(n_splits=K, shuffle=True, random_state=42).split(all_images):
    print(f"\n📂 Preparing Fold {fold_num}...")

    fold_dir = f"datasets/folds/fold{fold_num}"
    train_img_dir = os.path.join(fold_dir, "images/train")
    val_img_dir = os.path.join(fold_dir, "images/val")
    train_lbl_dir = os.path.join(fold_dir, "labels/train")
    val_lbl_dir = os.path.join(fold_dir, "labels/val")

    # Create directories
    for d in [train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir]:
        os.makedirs(d, exist_ok=True)

    # Copy training images and labels
    for idx in train_idx:
        img = all_images[idx]
        lbl = img.replace("images", "labels").replace(".jpg", ".txt")
        for ext in [".jpg", ".jpeg", ".png"]:
            lbl = lbl.replace(ext, ".txt")
        if os.path.exists(lbl):
            shutil.copy(img, train_img_dir)
            shutil.copy(lbl, train_lbl_dir)
        else:
            print(f"⚠️ Label not found for {img}, skipping...")

    # Copy validation images and labels
    for idx in val_idx:
        img = all_images[idx]
        lbl = img.replace("images", "labels").replace(".jpg", ".txt")
        for ext in [".jpg", ".jpeg", ".png"]:
            lbl = lbl.replace(ext, ".txt")
        if os.path.exists(lbl):
            shutil.copy(img, val_img_dir)
            shutil.copy(lbl, val_lbl_dir)
        else:
            print(f"⚠️ Label not found for {img}, skipping...")

    # Create fold-specific data.yaml with RELATIVE paths
    yaml_dict = {
        "train": "images/train",
        "val": "images/val",
        "nc": len(classes),
        "names": classes
    }
    yaml_path = os.path.join(fold_dir, "data.yaml")
    with open(yaml_path, "w") as f:
        yaml.dump(yaml_dict, f, sort_keys=False)

    # Create hyp.yaml with augmentations (ideal for YOLO11s)
    hyp_dict = {
        "hsv_h": 0.015,
        "hsv_s": 0.7,
        "hsv_v": 0.4,
        "degrees": 5.0,
        "translate": 0.1,
        "scale": 0.5,
        "shear": 0.0,
        "fliplr": 0.5,
        "flipud": 0.0,
        "mosaic": 1.0,
        "mixup": 0.1
    }

    hyp_path = os.path.join(fold_dir, "hyp.yaml")
    with open(hyp_path, "w") as f:
        yaml.dump(hyp_dict, f, sort_keys=False)

    print(f"✅ Fold {fold_num} ready! YAMLs: {yaml_path}, {hyp_path}")
    fold_num += 1


📂 Preparing Fold 1...
✅ Fold 1 ready! YAMLs: datasets/folds/fold1/data.yaml, datasets/folds/fold1/hyp.yaml

📂 Preparing Fold 2...
✅ Fold 2 ready! YAMLs: datasets/folds/fold2/data.yaml, datasets/folds/fold2/hyp.yaml

📂 Preparing Fold 3...
✅ Fold 3 ready! YAMLs: datasets/folds/fold3/data.yaml, datasets/folds/fold3/hyp.yaml

📂 Preparing Fold 4...
✅ Fold 4 ready! YAMLs: datasets/folds/fold4/data.yaml, datasets/folds/fold4/hyp.yaml

📂 Preparing Fold 5...
✅ Fold 5 ready! YAMLs: datasets/folds/fold5/data.yaml, datasets/folds/fold5/hyp.yaml


# Train Models

## Train YOLOv11 on Each Fold

In [ ]:
for k in range(1, K+1):
    print(f"\n🚀 Training Fold {k}...")

    hyp_path = f"datasets/folds/fold{k}/hyp.yaml"
    with open(hyp_path, "r") as f:
        hyp_params = yaml.safe_load(f)

    fold_model = YOLO("yolo11s.pt")

    fold_model.train(
        data=f"datasets/folds/fold{k}/data.yaml",
        epochs=50,
        imgsz=640,
        batch=16,
        project="runs/kfold",
        name=f"fold{k}",
        **hyp_params
    )


🚀 Training Fold 1...


Ultralytics 8.3.183 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/folds/fold1/data.yaml, degrees=5.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fold1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrain

Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

 22                  -1  1   1511424  ultralytics.nn.modules.block.C3k2            [768, 512, 1, True]           
 23        [16, 19, 22]  1    822891  ultralytics.nn.modules.head.Detect           [9, [128, 256, 512]]          
YOLO11s summary: 181 layers, 9,431,275 parameters, 9,431,259 gradients, 21.6 GFLOPs

Transferred 493/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 91.0±30.0 MB/s, size: 4915.8 KB)


...

train: Scanning /content/datasets/folds/fold4/labels/train... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:56<00:00,  3.52it/s]

train: /content/datasets/folds/fold4/images/train/0230f343-20250814_095032.jpg: corrupt JPEG restored and saved
train: /content/datasets/folds/fold4/images/train/0402f08a-20250814_095030.jpg: corrupt JPEG restored and saved
train: /content/datasets/folds/fold4/images/train/0676cecd-20250814_095415.jpg: corrupt JPEG restored and saved
train: /content/datasets/folds/fold4/images/train/0780e5ea-20250814_094325.jpg: corrupt JPEG restored and saved
train: /content/datasets/folds/fold4/images/train/078a527b-20250814_094825.jpg: corrupt JPEG restored and saved
train: /content/datasets/folds/fold4/images/train/07f7d780-20250814_101148.jpg: corrupt JPEG restored and saved
train: /content/datasets/folds/fold4/images/train/09b97a76-20250814_094457.jpg: corrupt JPEG restored and saved
train: /content/datasets/folds/fold4/images/train/09c38df8-20250814_094642.jpg: corrupt JPEG restored and saved
train: /content/datasets/folds/fold4/images/train/0a0cfaff-20250814_094331.jpg: corrupt JPEG restored an

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 169.4±14.3 MB/s, size: 4096.6 KB)


val: Scanning /content/datasets/folds/fold4/labels/val... 21 images, 1 backgrounds, 0 corrupt:  42%|████▏     | 21/50 [00:15<00:21,  1.36it/s]


KeyboardInterrupt: 

## Collect Metrics for Each Fold

In [ ]:
metrics_list = []

for k in range(1, K+1):
    results_csv = f"runs/kfold/fold{k}/results.csv"
    if os.path.exists(results_csv):
        df = pd.read_csv(results_csv)
        last_epoch = df.iloc[-1]
        metrics_list.append(last_epoch)
    else:
        print(f"⚠️ Results not found for Fold {k}")

metrics_df = pd.DataFrame(metrics_list)
print("\n📊 Metrics by Fold:")
print(metrics_df)

# Compute mean metrics
mean_metrics = metrics_df.mean()
print("\n✅ Mean Metrics Across Folds:")
print(mean_metrics)

# Export metrics
metrics_df.to_csv("kfold_results.csv", index=False)
mean_metrics.to_csv("kfold_mean.csv")

⚠️ Results not found for Fold 4
⚠️ Results not found for Fold 5

📊 Metrics by Fold:
    epoch     time  train/box_loss  train/cls_loss  train/dfl_loss  \
49   50.0  1331.91         0.33472         0.27349         0.81417   
49   50.0  1303.14         0.34188         0.27481         0.80552   
49   50.0  1552.92         0.33626         0.25553         0.80490   

    metrics/precision(B)  metrics/recall(B)  metrics/mAP50(B)  \
49               0.98263            0.85156           0.89112   
49               0.92716            0.90867           0.92414   
49               0.98579            0.83952           0.86700   

    metrics/mAP50-95(B)  val/box_loss  val/cls_loss  val/dfl_loss    lr/pg0  \
49              0.84692       0.31532       0.23557       0.80551  0.000023   
49              0.88129       0.30628       0.23063       0.79087  0.000023   
49              0.82378       0.31145       0.27702       0.80814  0.000023   

      lr/pg1    lr/pg2  
49  0.000023  0.000023  
49  0.0

## Evaluate and Select Best Model

In [ ]:
best_map = 0
best_fold = 0

for k in range(1, K+1):
    results_csv = f"runs/kfold/fold{k}/results.csv"
    if os.path.exists(results_csv):
        df = pd.read_csv(results_csv)

        map_cols = [c for c in df.columns if "mAP" in c]
        print(f"\n📊 Fold {k} - mAP columns: {map_cols}")

        if not map_cols:
            print(f"⚠️ No mAP metrics found in Fold {k}, skipping...")
            continue

        # Prefer mAP50-95 if available
        if any("50-95" in c for c in map_cols):
            col = [c for c in map_cols if "50-95" in c][0]
        else:
            col = map_cols[0]

        last_epoch_map = df[col].iloc[-1]
        print(f"Fold {k} - {col}: {last_epoch_map:.4f}")

        if last_epoch_map > best_map:
            best_map = last_epoch_map
            best_fold = k

if best_fold > 0:
    best_model_path = f"runs/kfold/fold{best_fold}/weights/best.pt"
    print(f"\n✅ Best model: Fold {best_fold} with mAP = {best_map:.4f}")
    print(f"Path: {best_model_path}")
else:
    print("❌ No valid fold found.")

best_model = YOLO(best_model_path)


📊 Fold 1 - mAP columns: ['metrics/mAP50(B)', 'metrics/mAP50-95(B)']
Fold 1 - metrics/mAP50-95(B): 0.8469

📊 Fold 2 - mAP columns: ['metrics/mAP50(B)', 'metrics/mAP50-95(B)']
Fold 2 - metrics/mAP50-95(B): 0.8813

📊 Fold 3 - mAP columns: ['metrics/mAP50(B)', 'metrics/mAP50-95(B)']
Fold 3 - metrics/mAP50-95(B): 0.8238

✅ Best model: Fold 2 with mAP = 0.8813
Path: runs/kfold/fold2/weights/best.pt


# Export Best Model

In [ ]:
onnx_path = best_model_path.replace(".pt", "_best.onnx")
best_model.export(format="onnx")
print(f"✅ Model exported to ONNX: {onnx_path}")

Ultralytics 8.3.183 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11s summary (fused): 100 layers, 9,416,283 parameters, 0 gradients, 21.3 GFLOPs

PyTorch: starting from 'runs/kfold/fold2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 13, 8400) (18.3 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.59', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 8.9s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.64...
ONNX: export success ✅ 11.6s, saved as 'runs/kfold/fold2/weights/best.onnx' (36.2 MB)

Export complete (12.9s)
Results saved to /content/runs/kfold/fold2/weights
Predict:         yolo predict tas